In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Lamar Jackson,BAL,QB,25,3,3,56,88,749,...,2,NaN,NaN,102,102.3,110.3,104.3,50,1,1
1,2,Nick Chubb,CLE,RB,27,3,3,0,0,0,...,4,NaN,NaN,61,64.9,67.9,62.9,35,1,2
2,3,Josh Allen,BUF,QB,26,3,3,94,132,1014,...,1,NaN,NaN,88,87.9,96.9,89.9,35,2,3
3,4,Tyreek Hill,MIA,WR,28,4,4,0,0,0,...,2,NaN,NaN,60,91.2,94.2,75.7,34,1,4
4,5,Jalen Hurts,PHI,QB,24,3,3,66,98,916,...,3,NaN,NaN,85,85.3,92.3,86.3,33,3,5


### 1. Get Current Fantasy Points

In [148]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Lamar Jackson,BAL,QB,3,102.3
1,Nick Chubb,CLE,RB,3,64.9
2,Josh Allen,BUF,QB,3,87.9
3,Tyreek Hill,MIA,WR,4,91.2
4,Jalen Hurts,PHI,QB,3,85.3
...,...,...,...,...,...
258,Albert Okwuegbunam,DEN,TE,3,10.5
263,Isaiah Likely,BAL,TE,3,10.1
264,Noah Fant,SEA,TE,3,14.4
274,Kylen Granson,IND,TE,3,11.3


### 2. Get Projections

##### 2a. My Projections

In [149]:
with open('projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Prob']]

In [150]:
proj[proj['Player'].str.contains("Etienne")]

,Player,Tm,FantPos,PrvPts_PPR,AverageDraftPositionPPR,Preds,Prob
35,Travis Etienne Jr.,JAX,RB,0.0,33.6,158.534834,0.498811


##### 2b. ESPN Projections

In [151]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}
games_played = {'ARI' : 3,
 'ATL' : 3,
 'BAL' : 3,
 'BUF' : 3,
 'CAR' : 3,
 'CHI' : 3,
 'CIN' : 4,
 'CLE' : 3,
 'DAL' : 3,
 'DEN' : 3,
 'DET' : 3,
 'GNB' : 3,
 'HOU' : 3,
 'IND' : 3,
 'JAX' : 3,
 'KAN' : 3,
 'LAC' : 3,
 'LAR' : 3,
 'LVR' : 3,
 'MIA' : 4,
 'MIN' : 3,
 'NWE' : 3,
 'NOR' : 3,
 'NYG' : 3,
 'NYJ' : 3,
 'PHI' : 3,
 'PIT' : 3,
 'SEA' : 3,
 'SFO' : 3,
 'TAM' : 3,
 'TEN' : 3,
 'WAS' : 3}


In [152]:
def getESPNprojs(link = './projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

In [154]:
newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
print(newerPts['espn'].value_counts())
newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
newerPts

newerPts['GP'] = newerPts['Tm'].replace(games_played)
newerPts['PPR_adj'] = newerPts['PPR'] * newerPts['GP']
newerPts.dtypes

             Player   Tm FantPos  G   PPR  PrvPts_PPR  \
127     Josh Palmer  LAC      WR  3  32.8         NaN   
115     Jeff Wilson  SFO      RB  3  28.9         NaN   
147     Taysom Hill  NOR      TE  2  16.7         NaN   
175  Zander Horvath  LAC      RB  3  15.9         NaN   
197  Reggie Gilliam  BUF      RB  3  13.9         NaN   
228     David Sills  NYG      WR  3  10.7         NaN   

     AverageDraftPositionPPR  Preds  Prob     _merge  
127                      NaN    NaN   NaN  left_only  
115                      NaN    NaN   NaN  left_only  
147                      NaN    NaN   NaN  left_only  
175                      NaN    NaN   NaN  left_only  
197                      NaN    NaN   NaN  left_only  
228                      NaN    NaN   NaN  left_only  
both          221
left_only      17
right_only      0
Name: espn, dtype: int64


Player                       object
Tm                           object
FantPos                      object
G                            object
PPR                         float64
PrvPts_PPR                  float64
AverageDraftPositionPPR     float64
Preds                       float64
Prob                        float64
_merge                     category
Position                     object
Points                      float64
espn                       category
GP                            int64
PPR_adj                     float64
dtype: object

In [155]:
newerPts['Points'].isnull().sum()

0

In [156]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.38212304747657005
0.40316751747468815


0      306.9
1      194.7
2      263.7
3      364.8
4      255.9
       ...  
233     31.5
234     30.3
235     43.2
236     33.9
237     30.3
Name: PPR_adj, Length: 238, dtype: float64